## 프로젝트 기반 빅데이터 서비스 솔루션 개발 전문 과정

#### 교과목명 : 모델 성능 평가
- 평가일 : 03.13
- 성명 : 권혁종
- 점수 : 70

Q1. iris data를 불러와서 붓꽃의 종류를 분류하는 모델링을 수행한 후 오차행렬과 정확도를 평가하세요.
- test_size = 0.2, 분류기는 DecisionTreeClassifier를 이용

In [2]:
import warnings
warnings.filterwarnings('ignore')
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.datasets import load_iris
data = load_iris()
dt = DecisionTreeClassifier(random_state=123)
X_train,X_test,y_train,y_test = train_test_split(data.data,data.target,test_size=0.2,random_state=123)
dt.fit(X_train,y_train)
pred=dt.predict(X_test)
pred_proba=dt.predict_proba(X_test)
con = confusion_matrix(y_test,pred)
acc = accuracy_score(y_test,pred)
print(f'혼동행렬 : \n {con} \n정확도 : {acc:0.4f}')

혼동행렬 : 
 [[13  0  0]
 [ 0  6  0]
 [ 0  1 10]] 
정확도 : 0.9667


Q2. 타이타닉 분석용 데이터세트인 tdf1.pkl를 불러와서 생존자 예측 모델을 만든 후 오차행렬, 정확도, 재현율, f1, AUC를 포함하는 사용자 함수를 활용하여 평가하세요.
- test_size = 0.2, 분류기는 RandomForestClassifier 이용

In [1]:
from sklearn.metrics import confusion_matrix,accuracy_score,precision_score,recall_score,f1_score,roc_auc_score
def scores(y_test,pred=None,pred_proba=None):
    con = confusion_matrix(y_test,pred)
    acc = accuracy_score(y_test,pred)
    pre = precision_score(y_test,pred)
    rec = recall_score(y_test,pred)
    f1 = f1_score(y_test,pred)
    roc = roc_auc_score(y_test,pred_proba)
    print(f'혼동행렬 : \n {con} \n정확도 : {acc:0.4f}, 정밀도 : {pre:0.4f}, 재현율 : {rec:0.4f}, f1_score : {f1:0.4f}, roc_auc : {roc:0.4f}')

In [25]:
from sklearn.ensemble import RandomForestClassifier
data = pd.read_pickle('./dataset/tdf1.pkl')
X = data.drop('Survived',axis=1)
y = data['Survived']
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=23)
rf = RandomForestClassifier(random_state=25)
rf.fit(X_train,y_train)
pred = rf.predict(X_test)
pred_proba = rf.predict_proba(X_test)[:,1]
scores(y_test,pred,pred_proba)

혼동행렬 : 
 [[102  13]
 [ 21  43]] 
정확도 : 0.8101, 정밀도 : 0.7679, 재현율 : 0.6719, f1_score : 0.7167, roc_auc : 0.8358


Q3. Q2에서 생성한 모델로 교차검증(cv=5)을 수행하고 평균 정확도를 출력하세요.

In [26]:
from sklearn.model_selection import cross_val_score
score = cross_val_score(dt,X,y,cv=5,scoring='accuracy')
score.mean()

0.7991023790094784

Q4. Q2에서 생성한 예측모델에 대하여 교차 검증 및 성능 개선을 수행하세요.(GridSearchCV 활용)

In [27]:
from sklearn.model_selection import GridSearchCV
params = {
    'max_depth' : [2,5,10],
    'min_samples_leaf' : [2,4,7]
}
grid = GridSearchCV(dt,param_grid=params,scoring='accuracy',cv=3,refit=True)
grid.fit(X_train,y_train)
pred = grid.best_estimator_.predict(X_test)
accuracy_score(y_test,pred)

0.8212290502793296

Q5 ~ Q7. 'dataset/diabetes.csv'을 불러와서 아래사항을 수행하세요.10
- 피마 인디언 당뇨병 예측을 로지스틱 회귀를 이용하여 수행하고 사용자 함수를 작성하여 평가(오차행렬, 정확도, 정밀도, 재현율, F1, ROC_AUC)
- 임곗값을 0.3에서 0.5까지 변화시키면서 정밀도와 재현율이 조정되는 과정을 시각화 
- 재현율 기준의 성능을 개선하기 위하여 그 값이 0이 될 수 없는 각 칼럼을 탐색하여 적절한 처리를 한 후 로지스틱 회귀로 예측 및 평가 수행(오차행렬, 정확도, 정밀도, 재현율, F1, ROC_AUC)


In [34]:
from sklearn.linear_model import LogisticRegression
data = pd.read_csv('./dataset/diabetes.csv')
lr = LogisticRegression()
X = data.drop('Outcome',axis=1)
y = data['Outcome']
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=23)
lr.fit(X_train,y_train)
pred = lr.predict(X_test)
pred_proba = lr.predict_proba(X_test)[:,1]
scores(y_test,pred,pred_proba)

혼동행렬 : 
 [[93  8]
 [18 35]] 
정확도 : 0.8312, 정밀도 : 0.8140, 재현율 : 0.6604, f1_score : 0.7292, roc_auc : 0.8670


Q8. "dataset/auto-mpg.xlsx"을 불러와서 회귀 모델을 생성하고 MSE, RMSE, R2로 평가를 수행하세요.

In [32]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error,r2_score
import numpy as np
mpg = pd.read_excel('./dataset/auto-mpg.xlsx')
mpg.drop('car name',axis=1,inplace=True)
mpg.horsepower.replace('?',np.nan,inplace=True)
mpg.dropna(subset='horsepower',inplace=True)
X = mpg.drop('mpg',axis=1)
y = mpg['mpg']
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=23)
lr = LinearRegression()
lr.fit(X_train,y_train)
pred = lr.predict(X_test)
mse = mean_squared_error(y_test,pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test,pred)
print(f'mse = {mse} , rmse = {rmse} , r2_score = {r2}')

mse = 12.62161417303934 , rmse = 3.5526911170321775 , r2_score = 0.7356060087557543


Q9. 'load_boston' 을 불러와서 cross_val_score를 이용한 cv=5인 교차검증을 수행 후 MSE, RMSE를 출력하세요.(LineaRegression) 

In [36]:
from sklearn.datasets import load_boston
data = load_boston()
lr = LinearRegression()
score = cross_val_score(lr,data.data,data.target,cv=5,scoring='neg_mean_squared_error')
print(f'평균 mse = {-1*score.mean()}, 평균 rmse = {np.sqrt(-1*score.mean())}')

평균 mse = 37.13180746769902, 평균 rmse = 6.093587405436884


Q10. 'Q9에 대하여 R2 Score를 구하세요.(k=5) 